In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from numpy.random import randn
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
data = pd.read_csv("C:/Users/HP/Desktop/dati sintetici/Escherichia_coli_Meropenem.csv", sep = '\t')
print (data.shape)
print (data.tail())
print (data.columns)

(205, 6002)
                                       ID      2000      2003      2006  \
200  47ef51be-3deb-4453-941c-08f3c84ccfcc  0.000472  0.000524  0.000650   
201  406818f0-c75a-49d2-bca7-4cdb74a17926  0.000479  0.000470  0.000194   
202  494d63a5-7879-4c7b-88e9-87cbece63710  0.000357  0.000161  0.000237   
203  e5fc5463-df08-4500-b3f5-d723832a5491  0.000532  0.000514  0.000731   
204  9d18bf07-46e5-4f4e-97fa-546f6b9dd589  0.000808  0.000809  0.000361   

         2009      2012      2015      2018      2021      2024  ...  \
200  0.000457  0.000448  0.000564  0.000569  0.000381  0.000159  ...   
201  0.000317  0.000426  0.000388  0.000566  0.000531  0.000343  ...   
202  0.000258  0.000205  0.000203  0.000237  0.000105  0.000041  ...   
203  0.000382  0.000134  0.000017  0.000082  0.000274  0.000338  ...   
204  0.000259  0.000258  0.000397  0.000328  0.000201  0.000082  ...   

        19973     19976     19979     19982         19985     19988     19991  \
200  0.000021  0.000033

In [16]:
#We will use all columns except the last column as features to train the model. Meropenem column will be used as the label of the model.
#features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
label = ['Meropenem']
X = data.drop(['Meropenem','ID'], 1)
y = data[label] 


data = (X.to_numpy()).reshape(X.shape[0], 6000).astype('float64')
#y = (y.to_numpy()).reshape(X.shape[0], 1).astype('')

C:\Users\HP\AppData\Local\Temp\ipykernel_16556\1372132325.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = data.drop(['Meropenem','ID'], 1)


In [47]:
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    
    return X, y

# generate n real samples with class labels; We randomly select n samples from the real data
def generate_real_samples(n):
    X1 = X.sample(n)
    #X = np.random.Generator.choice(data, size = n)   
    y = np.ones((n, 1))
    return X1, y

def define_generator(latent_dim, n_outputs):
    model = Sequential()
    model.add(Dense(15, activation='relu',  kernel_initializer='he_uniform', input_dim=latent_dim))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(n_outputs, activation='linear'))
    model.summary()
    return model

def define_discriminator(n_inputs):
    model = Sequential()
    model.add(Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #model.summary()
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [48]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=100, n_batch=20, n_eval=200):
    # determine half the size of one batch, for updating the  discriminator
    half_batch = int(n_batch / 2)
    d_history = []
    g_history = []
    # manually enumerate epochs
    for epoch in range(n_epochs):
    
        # prepare real samples
        x_real, y_real = generate_real_samples(half_batch)
        # prepare fake examples
        x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # update discriminator
        d_loss_real, d_real_acc = d_model.train_on_batch(x_real, y_real)
        d_loss_fake, d_fake_acc = d_model.train_on_batch(x_fake, y_fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        # prepare points in latent space as input for the generator
        x_gan = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = np.ones((n_batch, 1))
        # update the generator via the discriminator's error
        g_loss_fake = gan_model.train_on_batch(x_gan, y_gan)
        print('>%d, d1=%.3f, d2=%.3f d=%.3f g=%.3f' % (epoch+1, d_loss_real, d_loss_fake, d_loss,  g_loss_fake))
        d_history.append(d_loss)
        g_history.append(g_loss_fake)
        plot_history(d_history, g_history)
        g_model.save('trained_generated_model.h5')

In [51]:
# size of the latent space
latent_dim = 10
# create the discriminator
discriminator = define_discriminator(n_inputs=10)
# create the generator
generator = define_generator(latent_dim, n_outputs= 10)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_76 (Dense)            (None, 15)                165       
                                                                 
 dense_77 (Dense)            (None, 30)                480       
                                                                 
 dense_78 (Dense)            (None, 10)                310       
                                                                 
Total params: 955
Trainable params: 955
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 181ms/step


ValueError: in user code:

    File "C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\keras\engine\input_spec.py", l